In [3]:
import os
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

2023-11-10 02:10:22.538119: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-10 02:10:22.662821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 02:10:22.662851: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 02:10:22.663476: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 02:10:22.708891: I tensorflow/core/platform/cpu_feature_g

In [4]:
path = "image/base"
resize_path = 'image/resize'
folders = os.listdir(path)

In [5]:
classes = [f for f in folders if os.path.isdir(os.path.join(path, f))]
n_classes = len(classes)
n_classes

4

In [6]:
X = []
Y = []

In [7]:
for label, class_name in enumerate(classes):
    files = glob.glob(path + "/" + class_name + "/*.jpg")
    for file in files:
        img = cv2.imread(file)
        print(os.path.basename(file))
        img = cv2.resize(img, dsize=(224, 224))
        X.append(img)
        Y.append(label)

WIN_20231109_11_51_40_Pro.jpg
WIN_20231110_01_42_10_Pro (2).jpg
WIN_20231109_12_35_57_Pro.jpg
WIN_20231109_12_36_04_Pro.jpg
WIN_20231109_11_52_22_Pro.jpg
WIN_20231109_11_52_02_Pro.jpg
WIN_20231109_12_36_08_Pro.jpg
WIN_20231110_01_42_10_Pro.jpg
WIN_20231109_12_36_22_Pro.jpg
WIN_20231109_12_36_39_Pro.jpg
WIN_20231110_01_42_07_Pro (2).jpg
WIN_20231109_11_51_42_Pro.jpg
WIN_20231109_11_52_00_Pro.jpg
WIN_20231109_12_36_29_Pro.jpg
WIN_20231109_11_52_12_Pro.jpg
WIN_20231109_11_52_27_Pro.jpg
WIN_20231109_11_52_01_Pro.jpg
WIN_20231109_12_36_00_Pro.jpg
WIN_20231109_11_52_04_Pro.jpg
WIN_20231109_11_52_05_Pro (2).jpg
WIN_20231110_01_42_03_Pro (4).jpg
WIN_20231109_11_52_23_Pro.jpg
WIN_20231109_11_52_07_Pro.jpg
WIN_20231109_11_52_05_Pro.jpg
WIN_20231109_11_52_09_Pro (2).jpg
WIN_20231110_01_42_01_Pro.jpg
WIN_20231110_01_42_05_Pro (3).jpg
WIN_20231110_01_42_04_Pro.jpg
WIN_20231109_11_51_49_Pro.jpg
WIN_20231110_01_42_08_Pro.jpg
WIN_20231109_11_52_24_Pro.jpg
WIN_20231109_11_51_53_Pro.jpg
WIN_20231109_11_

In [8]:
X = np.array(X)
X = X.astype('float32')
X /= 255.0

In [9]:
Y = np.array(Y)
Y = to_categorical(Y, n_classes)
print(Y)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
# X_train = X
# Y_train = Y
print(X_train.shape)
print(Y_train.shape)

(268, 224, 224, 3)
(268, 4)


In [11]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import Model
from keras.layers import Input, Activation, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [12]:
input_tensor = Input(shape=(224,224,3))
base_model = VGG16(weights='imagenet', input_tensor=input_tensor, include_top=False)

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes, activation='softmax'))

model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

for layer in model.layers[:15]:
    layer.trainable = False
    
print('# layers=', len(model.layers))

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

model.summary()

2023-11-10 02:10:37.953005: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-10 02:10:37.964775: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-10 02:10:37.964964: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

58889256/58889256 [==============================] - 4s 0us/step
# layers= 20
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                 

In [14]:
model.fit(X_train, Y_train, epochs=50, batch_size=16)

score = model.evaluate(X_test, Y_test, batch_size=16)

Epoch 1/50
17/17 [==============================] - 1s 49ms/step - loss: 0.8730 - accuracy: 0.7836
Epoch 2/50
17/17 [==============================] - 1s 48ms/step - loss: 0.8333 - accuracy: 0.8097
Epoch 3/50
17/17 [==============================] - 1s 48ms/step - loss: 0.7965 - accuracy: 0.8134
Epoch 4/50
17/17 [==============================] - 1s 48ms/step - loss: 0.7611 - accuracy: 0.8358
Epoch 5/50
17/17 [==============================] - 1s 49ms/step - loss: 0.7275 - accuracy: 0.8545
Epoch 6/50
17/17 [==============================] - 1s 48ms/step - loss: 0.6951 - accuracy: 0.8619
Epoch 7/50
17/17 [==============================] - 1s 48ms/step - loss: 0.6639 - accuracy: 0.8769
Epoch 8/50
17/17 [==============================] - 1s 48ms/step - loss: 0.6358 - accuracy: 0.8918
Epoch 9/50
17/17 [==============================] - 1s 48ms/step - loss: 0.6075 - accuracy: 0.8993
Epoch 10/50
17/17 [==============================] - 1s 48ms/step - loss: 0.5813 - accuracy: 0.9067
Epoch 11/

In [15]:
import pickle
pickle.dump(classes, open('backend/models/classes.sav', 'wb'))
model.save('backend/models/cnn.h5')

/home/nakahira/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
from keras.models import load_model
import pickle
import cv2
import glob

In [17]:
model = load_model('backend/models/cnn.h5')
classes = pickle.load(open('backend/models/classes.sav', 'rb'))

In [18]:
img = cv2.imread('./image/base/front/front (2).jpg')
img = cv2.resize(img, dsize=(224, 224))
img = img.astype('float32')
img /= 255.0
img = img[None, ...]
result = model.predict(img)
pred = result.argmax()

img = cv2.imread('./image/base/front/front (2).jpg')
cv2.putText(img, classes[pred] + ' ' + str(result[0][pred] * 100) + '%', (0, 50), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 5, cv2.LINE_AA)
cv2.imwrite('./image/output/' + str(classes[pred]) + '/front (2).jpg', img)

1/1 [==============================] - 0s 293ms/step


True

In [20]:
files = glob.glob('./image/all/*')
for file in files:
    img = cv2.imread(file)
    img = cv2.resize(img, dsize=(224, 224))
    img = img.astype('float32')
    img /= 255.0
    img = img[None, ...]
    result = model.predict(img)
    pred = result.argmax()
    img = cv2.imread(file)
    cv2.putText(img, classes[pred] + ' ' + str(result[0][pred] * 100) + '%', (0, 50), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 5, cv2.LINE_AA)
    cv2.putText(img, classes[result.argmin()] + ' ' + str(result[0][result.argmin()] * 100) + '%', (0, 150), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 5, cv2.LINE_AA)
    cv2.imwrite('./image/output/' + str(classes[pred]) + '/' + os.path.basename(file), img)

1/1 [==============================] - 0s 12ms/step
